# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue


In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader

In [2]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

display_results (generic function with 1 method)

In [3]:
function Compute_AlphaAff(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 1 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end

function Compute_AlphaMax(alpha_step, v_k, dir_v_k)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 100/99 && all(v_k + (alpha+alpha_step) * dir_v_k .>= 0) 
        alpha += alpha_step
    end
    return alpha
end


Compute_AlphaMax (generic function with 1 method)

In [4]:

function MehrotraPCQuad(QM, max_iter, eps=1e-10, tol_step_x=1e-8, eps_mu=1e-8, alpha_step=1e-2, display=true)
    
    # get variables from QuadraticModel
    x_0 = QM.meta.x0
    @assert all(x_0 .> 0)
    A = jac(QM, x_0)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_0)
    c = QM.data.c
    c0 = QM.data.c0
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    s_0 = ones(n_cols)
    lambda_0 = Matrix(A)'\(c-s_0) # least square initialisation
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    
    rb_0 = A * x_0 - b
    rc_0 = -Q * x_0 + A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk, Sk = Diagonal(x_k), Diagonal(s_k)
    mu_k, rb_k, rc_k = copy(mu_0), copy(rb_0), copy(rc_0)
    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    
    
    while k<max_iter && opti_pdd==false && opti_rb==false && opti_rc==false && small_step_x==false && small_mu==false
        
        # Affine scaling direction
        Jacob_Fk = [-Q A' I
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk_aff = [-rc_k
                  -rb_k
                  -x_k.*s_k]
        dir_aff_k = Jacob_Fk\Fk_aff

        alpha_aff_pri = Compute_AlphaAff(alpha_step, x_k, dir_aff_k[1:n_cols])
        alpha_aff_dual = Compute_AlphaAff(alpha_step, s_k, dir_aff_k[n_rows+n_cols+1: end])
        mu_aff = (x_k + alpha_aff_pri * dir_aff_k[1:n_cols])' * 
                    (s_k + alpha_aff_dual * dir_aff_k[n_rows+n_cols+1: end]) / n_cols
        sigma = (mu_aff / mu_k)^3

        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e - dir_aff_k[1:n_cols].*dir_aff_k[n_rows+n_cols+1: end]] 
        dir_cc_k = Jacob_Fk\Fk_cc
        
        dir_k = dir_aff_k .+ dir_cc_k # final direction
         
        alpha_max_pri = Compute_AlphaMax(alpha_step, x_k, dir_k[1:n_cols])
        alpha_max_dual = Compute_AlphaMax(alpha_step, s_k, dir_k[n_rows+n_cols+1: end])
        
        # new parameters
        alpha_k_pri, alpha_k_dual = min(0.99*alpha_max_pri, 1), min(0.99*alpha_max_dual, 1)
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_k_dual * dir_k[n_cols+1: n_rows+n_cols]
        s_k += alpha_k_dual * dir_k[n_rows+n_cols+1: end]
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        
        k += 1
        
        if display == true
            @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuad (generic function with 6 methods)

In [5]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3];
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [6]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [7]:
res_mpc1 =  MehrotraPCQuad(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.75e+01    |         3.30e+00       |    1.12e+00  |    2.20e+00  |  0.00e+00  |  2.00e+00
   1 |      1.23e+01    |         3.22e-02       |    6.65e-02  |    7.64e-02  |  3.08e+00  |  2.51e-01
   2 |      1.35e+01    |         2.25e-03       |    1.98e-03  |    2.29e-03  |  2.40e-01  |  6.43e-03
   3 |      1.35e+01    |         2.45e-05       |    3.94e-05  |    4.66e-05  |  4.94e-03  |  9.70e-05
   4 |      1.35e+01    |         2.76e-07       |    7.85e-07  |    9.86e-07  |  9.82e-05  |  1.62e-06
   5 |      1.35e+01    |         3.38e-09       |    1.56e-08  |    2.04e-08  |  1.95e-06  |  2.92e-08
   6 |      1.35e+01    |         4.61e-11       |    3.11e-10  |    4.15e-10  |  3.89e-08  |  5.50e-10

 stopping criterion = ["pdd <= eps", "mu_k <= eps_mu"]

---------

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [5.1454e-10, 3.0, 5.97718e-10]
  "lambda_opt" => [-9.93575, 12.0]
  "s_opt"      => [1.93575, 3.12465e-11, 0.935748]
  "n_iter"     => 6
  "pdd"        => 4.61333e-11
  "cond_rb"    => 3.10886e-10
  "cond_rc"    => 4.14559e-10

In [9]:
### lp 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
res_mpc_verif2 =  MehrotraPCQuad(SM_verif2, 5);
display_results(res_mpc_verif2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.00e+00    |         8.33e-01       |    2.50e+00  |    4.08e-01  |  0.00e+00  |  2.00e+00
   1 |      4.85e-01    |         8.03e-01       |    2.50e-04  |    4.08e-05  |  3.28e+00  |  3.97e-01
   2 |      1.13e-01    |         1.15e-01       |    2.50e-08  |    5.66e-06  |  4.56e-01  |  4.28e-02
   3 |      2.06e-03    |         2.26e-03       |    4.98e-10  |    5.66e-10  |  1.37e-01  |  7.54e-04
   4 |      4.10e-05    |         4.30e-05       |    9.90e-12  |    5.66e-12  |  2.48e-03  |  1.43e-05

 stopping criterion = ["cond_rb <= eps", "cond_rc <= eps"]

-----------------------------------------------------------------------
------------------------------- RESULTS -------------------------------


OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [4.09655e-5, 0.432622, 0.567337]
  "lambda_opt" => [-2.04677e-6]
  "s_opt"      => [1.0, 2.04678e-6, 2.04678e-6]
  "n_iter"     => 4
  "pdd"        => 4.30105e-5
  "cond_rb"    => 9.90019e-12
  "cond_rc"    => 5.66238e-12

# Lecture des donnees .SIF

In [148]:
function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    x0 = zeros(length(qpdata.lvar))
    for i=1:length(x0)
        if qpdata.lvar[i] == -Inf && qpdata.uvar[i] == Inf
            x0[i] = 1.
        elseif qpdata.lvar[i] == -Inf && qpdata.uvar[i] != Inf
            x0[i] = qpdata.uvar[i] - 1.
        elseif qpdata.lvar[i] != -Inf && qpdata.uvar[i] == Inf
            x0[i] = qpdata.lvar[i] + 1.
        else
            x0[i] = (qpdata.lvar[i] + qpdata.uvar[i]) / 2 
        end
    end   
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0, name=name_pb)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end


displayQuadraticModel (generic function with 1 method)

In [116]:
path = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes"
afiro = string(path, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes\\AFIRO.SIF"

In [149]:
qpdata2 = readqps(afiro)
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2, "QM2")
SM2 = SlackModel(QM2)
displayQuadraticModel(SM2)

lvar = [0.0, 0.0, 0.0, 0.0, 0.0,

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf]
uvar = [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, 80.0, 0.0, 80.0, 0.0, 0.0, 0.0, 500.0, 0.0, 500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 310.0, 300.0]
x0 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [45]:
T = typeof(qpdata2)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname ncon is Int64
type of the fieldname objsense is Symbol
type of the fieldname c0 is Float64
type of the fieldname c is Array{Float64,1}
type of the fieldname qrows is Array{Int64,1}
type of the fieldname qcols is Array{Int64,1}
type of the fieldname qvals is Array{Float64,1}
type of the fieldname arows is Array{Int64,1}
type of the fieldname acols is Array{Int64,1}
type of the fieldname avals is Array{Float64,1}
type of the fieldname lcon is Array{Float64,1}
type of the fieldname ucon is Array{Float64,1}
type of the fieldname lvar is Array{Float64,1}
type of the fieldname uvar is Array{Float64,1}
type of the fieldname name is Union{Nothing, String}
type of the fieldname objname is Union{Nothing, String}
type of the fieldname rhsname is Union{Nothing, String}
type of the fieldname bndname is Union{Nothing, String}
type of the fieldname rngname is Union{Nothing, String}
type of the fieldname varnames is Array{String,1}
type of the 

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [354]:
function Compute_AlphaAff2(alpha_step, v_k, dir_v_k, lvar, uvar)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 1 && lvar <= (v_k + (alpha+alpha_step) * dir_v_k) <= uvar
        alpha += alpha_step
    end
    return alpha
end

function Compute_AlphaMax2(alpha_step, v_k, dir_v_k, lvar, uvar)
    alpha = 0
    n = length(v_k)
    while alpha+alpha_step <= 100/99 && lvar <= (v_k + (alpha+alpha_step) * dir_v_k) <= uvar
        alpha += alpha_step
    end
    return alpha
end

function Compute_mu(x_l, x_u, s, stilde, lvar, uvar)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s' * x_l + stilde' * x_u) / (length(x_l) + length(x_u))
end



Compute_mu (generic function with 2 methods)

In [407]:

function MehrotraPCQuad2(QM, max_iter, eps=1e-10, tol_step_x=1e-8, eps_mu=1e-8, alpha_step=1e-2, display=true)
    
    # get variables from QuadraticModel
    x_0 = QM.meta.x0
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    id_non_inf_lvar, id_non_inf_uvar = findall((x -> x!=-Inf), lvar), findall((x -> x!=Inf), uvar) # finite bounds index
    @assert all(x_0 .> lvar) && all(x_0 .< uvar)
    A = jac(QM, x_0)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_0)
    c = QM.data.c
    c0 = QM.data.c0
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    s_0, stilde_0 = zeros(n_cols), zeros(n_cols)
    s_0[id_non_inf_lvar] = ones(length(id_non_inf_lvar))
    stilde_0[id_non_inf_uvar] = ones(length(id_non_inf_uvar))
    lambda_0 = Matrix(A)'\(c -s_0+stilde_0) # least square initialisation, s_0 = stilde_0
    x_k, lambda_k, s_k, stilde_k = copy(x_0), copy(lambda_0), copy(s_0), copy(stilde_0)
    
    rb_0 = A * x_0 - b
    rc_0 = -Q * x_0 + A' * lambda_0 + s_0 - stilde_0 - c
    mu_0 = Compute_mu(x_0[id_non_inf_lvar], x_0[id_non_inf_uvar], 
                        s_0[id_non_inf_lvar], stilde_0[id_non_inf_uvar], 
                        lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])
    
    # matrices without infinity constraints
    nb_non_inf_l, nb_non_inf_u = length(id_non_inf_lvar), length(id_non_inf_uvar) # number of finite constraints
    nb_inf_l, nb_inf_u = n_cols-nb_non_inf_l, n_cols-nb_non_inf_u # number of infinite constraints
    Xk_non_inf_l = Diagonal(x_k[id_non_inf_lvar])
    Sk_non_inf = [Diagonal(s_k[id_non_inf_lvar])  zeros(nb_non_inf_l, nb_inf_l)] # add zeros to match number of cols
    Xk_non_inf_u = Diagonal(x_k[id_non_inf_uvar])
    Stildek_non_inf = [Diagonal(stilde_k[id_non_inf_uvar])  zeros(nb_non_inf_u, nb_inf_u)] 
    Lvar_non_inf = Diagonal(lvar[id_non_inf_lvar]) 
    Uvar_non_inf = Diagonal(uvar[id_non_inf_uvar]) 
    
    mu_k, rb_k, rc_k = copy(mu_0), copy(rb_0), copy(rc_0)
    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    

    while k<max_iter && opti_pdd==false && opti_rb==false && opti_rc==false && small_step_x==false && small_mu==false
        
        # Affine scaling direction
        Jacob_Fk = [-Q                      A'                     I(n_cols)[:, 1:nb_non_inf_l]       -I(n_cols)[:, 1:nb_non_inf_u]
                    A                zeros(n_rows, n_rows)         zeros(n_rows, nb_non_inf_l)        zeros(n_rows, nb_non_inf_u)
                    Sk_non_inf      zeros(nb_non_inf_l, n_rows)    Xk_non_inf_l-Lvar_non_inf         zeros(nb_non_inf_l, nb_non_inf_u)
                    Stildek_non_inf  zeros(nb_non_inf_u, n_rows)   zeros(nb_non_inf_u,nb_non_inf_l)   Xk_non_inf_u-Uvar_non_inf]
        Fk_aff = [-rc_k
                  -rb_k
                  -(x_k[id_non_inf_lvar]-lvar[id_non_inf_lvar]).*s_k[id_non_inf_lvar]
                  -(x_k[id_non_inf_uvar]-uvar[id_non_inf_uvar]).*stilde_k[id_non_inf_uvar]]
        
        dir_aff_k = Jacob_Fk\Fk_aff
        
        println(dir_aff_k)
        alpha_aff_pri = Compute_AlphaAff2(alpha_step, x_k, dir_aff_k[1:n_cols], lvar, uvar)
        alpha_aff_dual = Compute_AlphaAff(alpha_step, s_k[id_non_inf_lvar], 
                                            dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        println(alpha_aff_dual)
        alphatilde_aff_dual = Compute_AlphaAff(alpha_step, stilde_k[id_non_inf_uvar], 
                                                dir_aff_k[n_rows+n_cols+nb_non_inf_l+1:end])
        
        mu_aff = Compute_mu(x_k[id_non_inf_lvar] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_lvar],
                    x_k[id_non_inf_uvar] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_uvar],
                    s_k[id_non_inf_lvar] + alpha_aff_dual * dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l],
                    stilde_k[id_non_inf_uvar] + alphatilde_aff_dual * dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end],
                    lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])
        
        sigma = (mu_aff / mu_k)^3
        

        
        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e[1:nb_non_inf_l] - dir_aff_k[1:n_cols][id_non_inf_lvar].*dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
                 sigma*mu_k*e[1:nb_non_inf_u] - dir_aff_k[1:n_cols][id_non_inf_uvar].*dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end]] 
        
        dir_cc_k = Jacob_Fk\Fk_cc
        dir_k = dir_aff_k .+ dir_cc_k # final direction
        
        alpha_max_pri = Compute_AlphaMax2(alpha_step, x_k, dir_k[1:n_cols], lvar, uvar)
        alpha_max_dual = Compute_AlphaMax(alpha_step, s_k[id_non_inf_lvar], 
                                        dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        
        alphatilde_max_dual = Compute_AlphaMax(alpha_step, stilde_k[id_non_inf_uvar], 
                                            dir_k[n_rows+n_cols+nb_non_inf_l+1: end])
        
        # new parameters
        alpha_k_pri = min(0.99*alpha_max_pri, 1)
        alpha_k_dual = min(0.99*alpha_max_dual, 1)
        alphatilde_k_dual = min(0.99*alphatilde_max_dual, 1)
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_k_dual * dir_k[n_cols+1: n_rows+n_cols]
        s_k[id_non_inf_lvar] += alpha_k_dual * dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
        stilde_k[id_non_inf_uvar] += alphatilde_k_dual * dir_k[n_rows+n_cols+nb_non_inf_l+1: end]
        
        Xk_non_inf_l = Diagonal(x_k[id_non_inf_lvar])
        Sk_non_inf = [Diagonal(s_k[id_non_inf_lvar])  zeros(nb_non_inf_l, nb_inf_l)] 
        Xk_non_inf_u = Diagonal(x_k[id_non_inf_uvar])
        Stildek_non_inf = [Diagonal(stilde_k[id_non_inf_uvar])  zeros(nb_non_inf_u, nb_inf_u)] 
        
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = Compute_mu(x_k[id_non_inf_lvar], x_k[id_non_inf_uvar], 
                        s_k[id_non_inf_lvar], stilde_k[id_non_inf_uvar], 
                        lvar[id_non_inf_lvar], uvar[id_non_inf_uvar])
        
        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k - stilde_k - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2)) # test correct? ecart primal dual
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        
        k += 1
        
        if display == true
            @printf("% 4d |     % 7.2e    |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, "stilde_opt" => stilde_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuad2 (generic function with 7 methods)

In [410]:
res_mpc1 =  MehrotraPCQuad2(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.75e+01    |         3.30e+00       |    1.12e+00  |    2.20e+00  |  0.00e+00  |  2.00e+00
[-1.0947368421052635, 0.9052631578947365, -2.9052631578947365, -3.663157894736848, 12.789473684210527, 0.09473684210526345, -1.4526315789473683, -0.03157894736842115]
0.6800000000000004
   1 |      1.23e+01    |         3.22e-02       |    6.65e-02  |    7.64e-02  |  3.08e+00  |  2.51e-01
[-0.027120022959867356, 0.09187997704013165, -0.21087997704013053, -0.42471443393835084, 0.4323555673640554, 0.6346213912061244, -0.18591411721833437, -0.0417756263036694]
0.9600000000000006
   2 |      1.35e+01    |         2.20e-03       |    6.65e-06  |    8.69e-04  |  2.47e-01  |  4.09e-03
[-0.0027571984267363713, -0.002745298426736158, 0.0027333984267365373, 0.03187680961760974, -0.0095323028

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [3.17935e-9, 3.0, -3.15555e-9]
  "lambda_opt" => [-9.90771, 12.0]
  "s_opt"      => [1.90771, 3.12683e-9, 0.907705]
  "stilde_opt" => [0.0, 0.0, 0.0]
  "n_iter"     => 5
  "pdd"        => 2.22506e-9
  "cond_rb"    => 6.65228e-12
  "cond_rc"    => 8.69173e-10

In [409]:
res_mpc1 =  MehrotraPCQuad(SM, 1);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |      1.75e+01    |         3.30e+00       |    1.12e+00  |    2.20e+00  |  0.00e+00  |  2.00e+00
[-1.0947368421052635, 0.9052631578947365, -2.9052631578947365, -3.663157894736848, 12.789473684210527, 0.09473684210526345, -1.4526315789473683, -0.03157894736842115]
0.6800000000000004
   1 |      1.23e+01    |         3.22e-02       |    6.65e-02  |    7.64e-02  |  3.08e+00  |  2.51e-01

 stopping criterion = ["reached max_iter"]

-----------------------------------------------------------------------
------------------------------- RESULTS -------------------------------


OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [0.0174702, 2.89847, 0.22053]
  "lambda_opt" => [-9.41673, 11.5058]
  "s_opt"      => [1.14892, 0.180202, 0.954706]
  "n_iter"     => 1
  "pdd"        => 0.0322389
  "cond_rb"    => 0.066523
  "cond_rc"    => 0.0763961

In [560]:
@benchmark MehrotraPCQuad(SM, 20, 1e-10, 1e-8, 1e-8, 1e-2, false)

BenchmarkTools.Trial: 
  memory estimate:  944.25 KiB
  allocs estimate:  10650
  --------------
  minimum time:     632.101 μs (0.00% GC)
  median time:      646.800 μs (0.00% GC)
  mean time:        761.476 μs (13.72% GC)
  maximum time:     6.606 ms (87.67% GC)
  --------------
  samples:          6534
  evals/sample:     1

In [173]:
a = [Inf; Inf;0;0;3;4; Inf]
findall((x -> x!=Inf), a)

4-element Array{Int64,1}:
 3
 4
 5
 6

In [237]:
b = [1,2,3,4,5,6,7]
b[3:6][1:2]

2-element Array{Int64,1}:
 3
 4

In [286]:
[I(2)

2×2 Diagonal{Bool,Array{Bool,1}}:
 1  ⋅
 ⋅  1